## Tensors

- Torch tensors basically share all the basic numpy array functionality (slicing, mutability, shape, etc), and for all intents and purposes you can treat a tensor like a numpy array

In [2]:
import torch
import numpy as np

### Initialising a Tensor

In [15]:
data = [[1, 2],[3, 4]]
numpy_array = np.array(data)
tensor_from_raw_data = torch.tensor(data)
tensor_from_numpy_array = torch.tensor(numpy_array)
tensor_from_another_tensor = torch.rand_like(tensor_from_numpy_array, dtype=torch.float) ## follows the shape of the provided tensor
tensor_with_random_values_but_fixed_shape = torch.rand((2,2))
tensor_with_ones_but_fixed_shape = torch.ones((2,2))
tensor_with_zeroes_but_fixed_shape = torch.zeros((2,2))

### Tensor Attributes

- Like a numpy array, a tensor has a `shape`, and a `dtype`
- Unique to torch, a tensor also has a `device` attribute. This indicates where the tensor is stored. If you want to do training, make sure your tensors are on the right device (e.g. CUDA), or you won't get the speed up

In [19]:
(
    tensor_from_raw_data.shape
    , tensor_from_raw_data.dtype
    , tensor_from_raw_data.device
    , tensor_from_raw_data.to(torch.accelerator.current_accelerator()).device
)

(torch.Size([2, 2]),
 torch.int64,
 device(type='cpu'),
 device(type='mps', index=0))

### Tensor operations

#### Slicing

In [54]:
tensor = torch.ones((4,4))
first_row = tensor[0]
first_col = tensor[:,0]
last_col = tensor[:,-1]
first_row, first_col, last_col

(tensor([1., 1., 1., 1.]), tensor([1., 1., 1., 1.]), tensor([1., 1., 1., 1.]))

#### Reassignment

In [55]:
tensor[:, 0] = 0
tensor

tensor([[0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 1.]])

#### Concatenation

In [56]:
torch.cat([tensor, tensor], dim=0) ## dim=0 by default, rowwise concatenation
torch.cat([tensor, tensor], dim=1) 

tensor([[0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.]])

#### Arithmetic Operations

In [57]:
tensor = torch.cat([torch.ones((1,4)), torch.ones((1,4))*2, torch.ones((1,4))*3, torch.ones((1,4))*4])

matmul1 = tensor @ tensor.T
matmul2 = tensor.matmul(tensor.T)
assert torch.equal(matmul1, matmul2)

element_product1 = tensor * tensor
element_product2 = tensor.mul(tensor)
assert torch.equal(element_product1, element_product2)

#### Retrieve value in single element tensor

In [58]:
tensor = torch.cat([torch.ones((1,4)), torch.ones((1,4))*2, torch.ones((1,4))*3, torch.ones((1,4))*4])
# tensor.item() ##Error, because tensor has multiple elements
tensor.sum().item()

40.0

#### In-place tensor operations

In [59]:
tensor = torch.cat([torch.ones((1,4)), torch.ones((1,4))*2, torch.ones((1,4))*3, torch.ones((1,4))*4])
tensor.add_(1)

tensor([[2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [4., 4., 4., 4.],
        [5., 5., 5., 5.]])

### Torch tensors <--> Numpy Interoperability

- Torch tensors are interoperable with numpy via numpy bridge

- WITH 1 BIG CAVEAT: when "converting" from torch tensor to numpy array, you are not actually making a copy. Instead, you create 2 pointers that point to the same data location in memory

In [65]:
torch_tensor = torch.rand((2,5))

In [66]:
tensor_to_numpy = torch_tensor.numpy() 
numpy_to_tensor = torch.from_numpy(tensor_to_numpy)

tensor_to_numpy, numpy_to_tensor

(array([[0.8206627 , 0.0058443 , 0.47440398, 0.5105605 , 0.7457159 ],
        [0.9837789 , 0.5171297 , 0.29087025, 0.90522856, 0.567249  ]],
       dtype=float32),
 tensor([[0.8207, 0.0058, 0.4744, 0.5106, 0.7457],
         [0.9838, 0.5171, 0.2909, 0.9052, 0.5672]]))

In [67]:
torch_tensor[0] = 0
tensor_to_numpy, numpy_to_tensor

(array([[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.9837789 , 0.5171297 , 0.29087025, 0.90522856, 0.567249  ]],
       dtype=float32),
 tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9838, 0.5171, 0.2909, 0.9052, 0.5672]]))